In [ ]:
import re
from typing import List, Tuple

import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gensim
from gensim.models import Word2Vec, FastText
from tqdm.notebook import tqdm

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from src.dataset import read_lang_dataset, tokenize_dataset, tokenize_document, get_vocab_mapping, gateways

sns.set_theme()
sns.set(rc={'figure.figsize': (12, 8)})

In [ ]:
dataset = read_lang_dataset('data/dataset_github_codes.db')
dataset

In [ ]:
dataset.language.value_counts()

In [ ]:
use_cache = True
# load "tokenized_sample.pkl" from disk if it exists
if use_cache and os.path.exists("models/tokenized_sample.pkl"):
    tokenized_sample = pd.read_pickle("models/tokenized_sample.pkl")
else:
    tokenized_sample = tokenize_dataset(dataset)
    tokenized_sample.to_pickle("models/tokenized_sample.pkl")

tokenized_sample

In [ ]:
def create_lang_model(dataset, complete_model, language):
    model = Word2Vec.load("models/complete_model.gensim")
    #model = FastText.load("models/complete_model.gensim")
    #model = Word2Vec(vector_size=100, window=10, min_count=10, workers=4)
    #model.reset_from(complete_model)
    #model.init_weights()
    train_dataset = dataset.code[dataset.language == language]
    model.train(train_dataset, total_examples=len(train_dataset), epochs=5)
    return model

In [ ]:
common_keywords = ["if", "else", "for", "while", "class", "struct", "return", "continue", "break", "var", "try", "catch", "except"]
semantic_symbols = ["{", "}", ":"]

language_specific_words = common_keywords + semantic_symbols

In [ ]:
from functools import partial


def add_language_prefix(code, language):
    return [f"{language}_{keyword}" if keyword in language_specific_words else keyword for keyword in code]

for language in tokenized_sample.language.unique():
    tokenized_sample.loc[tokenized_sample.language == language, "code"] = tokenized_sample[tokenized_sample.language == language].code.apply(partial(add_language_prefix, language=language))

In [ ]:
#complete_model = FastText(vector_size=100, window=5, min_count=5, workers=8)
#complete_model = Word2Vec(vector_size=500, window=5, min_count=10, workers=8)
complete_model = FastText(vector_size=100, window=5, min_count=10, workers=8)
complete_model.build_vocab(tokenized_sample.code)
complete_model.train(tokenized_sample.code, total_examples=complete_model.corpus_count, epochs=5)
complete_model

# Visualize

In [ ]:
inserted_tokens = ["INT_LITERAL", "FLOAT_LITERAL", "STRING_LITERAL", "VARIABLE"]
math_ops = ["+", "-", "*", "/", "%", "&&", "||", "!", "&", "|"]

python_keywords = ["def", "class", "return", "if", "else", "for", "while", "in", "import", "from", "as", "with", "try", "except", "finally", "raise", "assert", "yield", "lambda", "pass", "break", "continue", "del", "global", "nonlocal", "and", "or", "not", "is", "in", "True", "False", "None", "async", "await"]
cpp_keywords = ["class", "bool", "catch", "try", "break", "continue", "delete", "do", "else", "enum", "explicit", "export", "extern", "false", "for", "friend", "goto", "if", "inline", "mutable", "namespace", "new", "operator", "private", "protected", "public", "register", "return", "sizeof", "static", "struct", "switch", "template", "this", "throw", "true", "typedef", "typeid", "typename", "union", "using", "virtual", "volatile", "while"]
csharp_keywords = ["class", "bool", "catch", "try", "break", "continue", "delete", "do", "else", "enum", "explicit", "export", "extern", "false", "for", "friend", "goto", "if", "inline", "mutable", "namespace", "new", "operator", "private", "protected", "public", "register", "return", "sizeof", "static", "struct", "switch", "template", "this", "throw", "true", "typedef", "typeid", "typename", "union", "using", "virtual", "volatile", "while"]
go_keywords = ["break", "func", "default", "type", "defer", "go", "struct", "map", "chan", "else", "goto", "package", "range", "const", "fallthrough", "for", "import", "interface", "return", "select", "case", "continue", "if", "switch", "var", "nil", "true", "false"]

In [ ]:
def word_cloud(model: Word2Vec, words):
    words = [word for word in words if word in model.wv.key_to_index.keys()]
    word_vectors = np.array([model.wv.get_normed_vectors()[model.wv.key_to_index[word]] for word in words])
    pca = PCA(n_components=2)
    result = pca.fit_transform(word_vectors)

    df = pd.DataFrame(result, columns=["x", "y"])
    df["model"] = [word.split("_")[0] if len(word.split("_")) == 2 else "" for word in words]
    ax = sns.scatterplot(data=df, x="x", y="y", hue="model")
    range_x, range_y = np.ptp(df[["x", "y"]], axis=0)
    keywords = [word.split("_")[1] if len(word.split("_")) == 2 else word for word in words]
    for i, word in enumerate(keywords):
        x, y = result[i]
        ax.text(x + 0.005 * range_x, y, word, verticalalignment='center', horizontalalignment='left', fontsize=8)
    return ax

In [ ]:
word_cloud(complete_model, [f"{language}_{w}" for w in common_keywords for language in tokenized_sample.language.unique()] + ["foreach", 'dataclass'])

In [ ]:
complete_model.wv.most_similar(positive=["np", ".", "array", "Go", "VARIABLE"], negative=["Python", "C++", "C#"])

In [ ]:
python_codes = tokenized_sample[tokenized_sample["language"] == 'Python']

python_model = FastText(vector_size=100, window=5, min_count=10, workers=8, sg=1)
python_model.build_vocab(python_codes.code)
python_model.train(python_codes.code, total_examples=python_model.corpus_count, epochs=5)
python_model


In [ ]:
python_code_example = """#!/usr/bin/env python3

import os

def main():
    query = 7
    context = [3, 4, 10, 2, 8, 6, 5, 9, 7, 1]
    for i in range(len(context)):
        if context[i] == query:
            print(f'Query found at index {i}')
            os.exit(0)    

    print(f'Coulnd't find query {query}')
    os.exit(-1)


if __name__ == '__main__':
    main()

}"""

tokenized_code_example = tokenize_document(python_code_example, "Python")
for word in tokenized_code_example:
    print(f' {word}', end="")

def next_similar_token(token: str) -> str:
    next_most_similar_token, _ = complete_model.wv.most_similar(positive=[token])[0]
    return next_most_similar_token



next_similar_document = ' '.join([next_similar_token(t) for t in tokenized_code_example])
next_similar_document = next_similar_document.replace(':', ':\n')
print(next_similar_document)

complete_model.wv.most_similar(positive=['import', 'C++'], negative=['Python'])


In [ ]:
recent_tokens = ['import', 'os']

#python_model.wv.most_similar_to_given

for i in range(20):
    next_tokens = python_model.predict_output_word(recent_tokens[-5:])

    token_found = False
    for t, _ in next_tokens:
        if t in recent_tokens[-5:] and t not in ['import', ',']:
            continue

        recent_tokens.append(t)
        print(t, end=' ')
        token_found = True

    if not token_found:
        t, _ = next_tokens[0]
        recent_tokens.append(t)
        print(t, end=' ')


In [ ]:
colors = ["red", "blue", "green", "black"]

for language in tokenized_sample.language.unique():
    code_for_language = tokenized_sample[tokenized_sample["language"] == language]
    token_frequency = {}
    for code in code_for_language.code:
        for token in code:
            token_frequency[token] = token_frequency.get(token, 0) + 1
    print(f'Number to different tokens: {len(token_frequency)}')

    token_frequency_meta = {}
    for m in [10,20,30,40,50,60,70,80,90,100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]:
        for token, freq in token_frequency.items():
            if freq >= m:
                token_frequency_meta[m] = token_frequency_meta.get(m, 0) + 1
    plt.plot(list(token_frequency_meta.keys()), list(token_frequency_meta.values()), label=language)

plt.title('Vocabulary size by minimum token frequency')
plt.xlabel('Minimum token frequency')
plt.ylabel('Vocabulary size')
plt.legend()
plt.show()